In [1]:
import pysal
import numpy
import scipy
import pandas
import shapefile as shp
from sklearn import tree
from pymining import seqmining
import matplotlib.pyplot as plt
from sklearn import preprocessing

%matplotlib inline
plt.style.use('ggplot')
pandas.set_option('display.max_rows', 100, "display.max_columns", 100)

In [2]:
db_business = pysal.open('./dataset/Data/CLT_Business_Attri.dbf','r')
df_business = pandas.DataFrame(db_business[:], columns=db_business.header)

db_crimes = pysal.open('./dataset/Data/CrimeIncident_CMPD_2010.dbf','r')
df = pandas.DataFrame(db_crimes[:], columns=db_crimes.header)
df.shape

(67595, 53)

In [3]:
street2block = df_business.set_index('ST_NAME')["Block_id"].to_dict()
df["BLOCKID10"] = df["Street_Nam"].map(lambda street: street2block.get(street, "not found"))

df = df[df["BLOCKID10"] != "not found"]
df.shape

(50455, 54)

In [ ]:
encoder = preprocessing.LabelEncoder()
encoder.fit(list(set(df["NIBRSclass"])))

In [24]:
# only number and class
date_dict = dict()
for i, row in df.iterrows():
    date = row["Report_Dat"][:-5]
    t = (row["Report_Dat"][-4:], row["NIBRSclass"], row["BLOCKID10"])
    date_dict[date] = date_dict.get(date, []) + [t]
    
date_dict = {k: sorted(date_dict[k], key=lambda x: x[0]) for k in date_dict}
sequences_3 = [ [v[1] for v in date_dict[k] if v[2][-4] == "3"] for k in sorted(date_dict.keys())]

In [33]:
type(sequences_3[0][0])

unicode

In [36]:
with open("./dataset/gino.txt", "w+") as output:
    for day in sequences_3:
        for crime in day[:-2]:
            output.write(crime + ",")
        output.write(day[-1] + "\n")

In [30]:
%%time
# seqs = [ ["a",2,3], [3,4,5], ["a",2,3], [2,3,4,5] ]
freq_seqs = seqmining.freq_seq_enum(sequences_3, 100)
print len(sequences_3)
print len(freq_seqs)
sorted(freq_seqs)

365
484
CPU times: user 407 ms, sys: 5.19 ms, total: 412 ms
Wall time: 411 ms


In [39]:
freq_seqs

{((u'Aggravated Assault',), 162),
 ((u'Aggravated Assault', u'Burglary/B&E'), 107),
 ((u'All Other Offenses',), 280),
 ((u'All Other Offenses', u'All Other Offenses'), 153),
 ((u'All Other Offenses', u'All Other Thefts'), 197),
 ((u'All Other Offenses', u'All Other Thefts', u'All Other Thefts'), 124),
 ((u'All Other Offenses', u'All Other Thefts', u'Burglary/B&E'), 136),
 ((u'All Other Offenses', u'All Other Thefts', u'Drug/Narcotic Violations'),
  100),
 ((u'All Other Offenses', u'All Other Thefts', u'Simple Assault'), 130),
 ((u'All Other Offenses', u'All Other Thefts', u'Theft From Motor Vehicle'),
  127),
 ((u'All Other Offenses', u'Burglary/B&E'), 205),
 ((u'All Other Offenses', u'Burglary/B&E', u'All Other Thefts'), 112),
 ((u'All Other Offenses', u'Burglary/B&E', u'Burglary/B&E'), 136),
 ((u'All Other Offenses', u'Burglary/B&E', u'Simple Assault'), 121),
 ((u'All Other Offenses', u'Burglary/B&E', u'Theft From Motor Vehicle'), 120),
 ((u'All Other Offenses', u'Damage/Vandalism Of

In [ ]:
!java -jar aprioriallcod.jar "/Users/chris/Workspace/app/crime-analysis/dataset/gino.txt" 10 "./prova.txt" FREQUENT